In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from typing import Optional, List, Dict, Union
from datetime import datetime, timedelta, timezone
from bson.objectid import ObjectId

load_dotenv()
API_KEY = os.environ.get("ALPACA_KEY")
ALPACA_SECRET = os.environ.get("ALPACA_SECRET")
ALPACA_AUTH = os.environ.get("ALPACA_AUTH")

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": ALPACA_AUTH
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
clients_account_id = {"client_1": "dcfbefb1-cbaa-4e60-9cef-9db801b8eb13",
                      "client_2": "edafa6db-b501-4d69-95c8-b92793a3680c"}

In [22]:
def get_account_trading_details(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/account"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

Make INCOMING (flowing into the wallet) simulated bank transaction for the client into his account wallet. 
1. Get the client-bank relationship_id.

In [23]:
account_id = "dcfbefb1-cbaa-4e60-9cef-9db801b8eb13"

url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/ach_relationships"

response = requests.get(url, headers=headers)

print(response.json())
relationship_id = response.json()[0]["id"]
print(f"The relationship_id is {id}")

[{'id': 'd69ab9ad-a76f-472c-ab2f-ce9f33c240f2', 'account_id': 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13', 'created_at': '2025-01-14T19:37:28.811463Z', 'updated_at': '2025-01-14T19:39:49.308484Z', 'status': 'APPROVED', 'account_owner_name': 'Clever Wilson', 'bank_account_type': 'CHECKING', 'bank_account_number': '32131231abc', 'bank_routing_number': '123103716', 'nickname': 'Bank of America Checking', 'processor_token': None}]
The relationship_id is <built-in function id>


In [24]:
account_id = "dcfbefb1-cbaa-4e60-9cef-9db801b8eb13"

def get_account_relationship_id(account_id: str):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/ach_relationships"

    response = requests.get(url, headers=headers)

    relationship_id = response.json()[0]["id"]
    return relationship_id


relationship_id = get_account_relationship_id(clients_account_id["client_1"])

print(f"The relationship_id for client '{clients_account_id['client_1']}' is '{relationship_id}'.")

The relationship_id for client 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13' is 'd69ab9ad-a76f-472c-ab2f-ce9f33c240f2'.


2. Make the transaction (max. 1 per day)

In [25]:
url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/transfers"

payload = {
    "transfer_type": "ach",
    "direction": "INCOMING",
    "timing": "immediate",
    "relationship_id": f"{relationship_id}",
    "amount": "10"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": ALPACA_AUTH
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())



{'code': 40010001, 'message': 'maximum total daily transfer allowed is $50000'}


In [26]:
def make_funding_transaction(account_id: str, relationship_id, amount):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/accounts/{account_id}/transfers"

    payload = {
        "transfer_type": "ach",
        "direction": "INCOMING",
        "timing": "immediate",
        "relationship_id": f"{relationship_id}",
        "amount": str(amount)
    }

    response = requests.post(url, json=payload, headers=headers)

    print(response.json())

    return response.json()

make_funding_transaction(clients_account_id['client_1'], "d69ab9ad-a76f-472c-ab2f-ce9f33c240f2", 50000)

{'code': 40010001, 'message': 'maximum total daily transfer allowed is $50000'}


{'code': 40010001, 'message': 'maximum total daily transfer allowed is $50000'}

In [27]:
def get_open_positions_client(account_id: str, symbol_id: Optional[str] = None):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions"

    if symbol_id:
        url += f"/{symbol_id}"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

In [28]:
def close_all_open_positions_client(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions"

    response = requests.delete(url, headers=headers)

    print(response.json())

    return response.json()

In [29]:
def close_symbol_position_client(account_id: str, symbol: str, magnitude: float, mode: str="percentage"):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions/{symbol}"

    if mode == "percentage":
        url += f"?percentage={magnitude}"
    elif mode == "qty":
        url += f"?qty={magnitude}"
    else:
        raise ValueError("The mode especified is incorrect. It should be either 'percentage' or 'qty'")

    response = requests.delete(url, headers=headers)

    print(response.json())

    return response.json()

In [30]:
def list_open_positions_by_client_id(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/positions"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

In [296]:
def get_order_by_id(account_id, order_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders/{order_id}"

    response = requests.get(url, headers=headers)

    print(response.json())

    return response.json()

In [290]:
def get_list_of_orders(
    account_id: str,
    status: Optional[str] = "all",
    limit: Optional[int] = None,
    after: Optional[Union[str, datetime]] = None,
    until: Optional[Union[str, datetime]] = None,
    direction: Optional[str] = None,
    nested: Optional[bool] = None,
    symbols: Optional[str] = None,
    qty_below: Optional[Union[str, float]] = None,
    qty_above: Optional[Union[str, float]] = None,
) -> Dict:
    """
    Retrieve a list of orders for a specific account with optional filters.

    Args:
        account_id (str): The ID of the account for which to retrieve orders.
        status (str, optional): Order status to be queried. Can be 'open', 'closed', or 'all'. Defaults to 'open'.
        limit (int, optional): The maximum number of orders in the response. Defaults to 50, max is 500.
        after (Union[str, datetime], optional): A timestamp in ISO 8601 format or a datetime object. 
            The response will include only orders submitted after this timestamp (exclusive).
        until (Union[str, datetime], optional): A timestamp in ISO 8601 format or a datetime object. 
            The response will include only orders submitted until this timestamp (exclusive).
        direction (str, optional): The chronological order of the response based on submission time. 
            Can be 'asc' or 'desc'. Defaults to 'desc'.
        nested (bool, optional): If True, the result will roll up multi-leg orders under the 'legs' field of the primary order.
        symbols (str, optional): A comma-separated list of symbols to filter by (e.g., 'AAPL,TSLA').
        qty_below (Union[str, float], optional): Filter orders with a quantity below this value.
        qty_above (Union[str, float], optional): Filter orders with a quantity above this value.

    Returns:
        dict: A JSON response containing the list of orders matching the specified filters.
    """
    
    # Function implementation here
    pass
    # Base URL for the Alpaca Broker API
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders?"

    # Add status if provided
    if status:
        url += f"&status={status}"

    # Add limit if provided
    if limit:
        url += f"&limit={limit}"

    # Add after timestamp if provided
    if after:
        url += f"&after={after}"

    # Add until timestamp if provided
    if until:
        url += f"&until={until}"

    # Add direction if provided
    if direction:
        url += f"&direction={direction}"

    # Add nested flag if provided
    if nested:
        url += f"&nested={nested}"

    # Add symbols if provided (comma-separated list)
    if symbols:
        url += f"&symbols={symbols}"

    # Add qty_below if provided
    if qty_below:
        url += f"&qty_below={qty_below}"

    # Add qty_above if provided
    if qty_above:
        url += f"&qty_above={qty_above}"


    response = requests.get(url, headers=headers)

    return response.json()

orders = get_list_of_orders(
    account_id="your_account_id",
    status="open",
    limit=100,
    after="2021-01-01T18:38:00",
    until="2021-03-01T18:38:00",
    direction="desc",
    nested=True,
    symbols="AAPL,TSLA",
    qty_below="80",
    qty_above="100"
)
print(orders)

{'code': 40010001, 'message': 'invalid account id'}


In [33]:
def cancel_all_open_orders(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders"

    response = requests.delete(url, headers=headers)

    print(response.json())

In [34]:
def cancel_open_order(account_id, order_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders/{order_id}"

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"Order '{order_id}' cancelled succesfully.")
    else:
        print("Request error.")


In [35]:
def create_order_for_client(
    account_id: str,
    symbol: str,
    qty: Optional[Union[str, float]] = None,
    notional: Optional[Union[str, float]] = None,
    side: str = "buy",
    type: str = "market",
    time_in_force: str = "day",
    limit_price: Optional[Union[str, float]] = None,
    stop_price: Optional[Union[str, float]] = None,
    trail_price: Optional[Union[str, float]] = None,
    trail_percent: Optional[Union[str, float]] = None,
    extended_hours: bool = False,
    client_order_id: Optional[str] = None,
    order_class: str = "simple",
    tags: Optional[List[str]] = None,
    take_profit: Optional[Dict] = None,
    stop_loss: Optional[Dict] = None,
    commission: Optional[float] = None,
    commission_type: str = "notional",
    source: Optional[str] = None,
    instructions: Optional[str] = None,
    subtag: Optional[str] = None,
    swap_fee_bps: Optional[Union[str, float]] = None,
    position_intent: Optional[str] = None,
) -> Dict:
    """
    Create an order for a client's account.

    Args:
        account_id (str): The ID of the account for which to create the order.
        symbol (str): Symbol or asset ID to identify the asset to trade.
        qty (str, optional): Number of shares to trade. Can be fractional for market and day orders.
        notional (str, optional): Dollar amount to trade. Cannot be used with qty.
        side (str, optional): Represents the side of the transaction. Defaults to "buy".
        type (str, optional): The order type. Defaults to "market". Equity assets have the following options: limit, stop, stop_limit, trailing_stop.
        time_in_force (str, optional): The time the order will last without being fulfilled until cancellation. Defaults to "day".
        limit_price (str, optional): Required if type is 'limit' or 'stop_limit'.
        stop_price (str, optional): Required if type is 'stop' or 'stop_limit'.
        trail_price (str, optional): Required if type is 'trailing_stop' and trail_percent is not provided.
        trail_percent (str, optional): Required if type is 'trailing_stop' and trail_price is not provided.
        extended_hours (bool, optional): If true, order is eligible to execute in pre/post-market. Defaults to False.
        client_order_id (str, optional): A unique identifier for the order. Automatically generated if not provided.
        order_class (str, optional): The order class. Defaults to "simple".
        tags (list, optional): List of order tags (max 4).
        take_profit (dict, optional): Take profit order details.
        stop_loss (dict, optional): Stop loss order details.
        commission (float, optional): The commission to collect from the user.
        commission_type (str, optional): How to interpret the commission value. Defaults to "notional".
        source (str, optional): Source of the order.
        instructions (str, optional): Additional instructions for the order.
        subtag (str, optional): Subtag for the order.
        swap_fee_bps (str, optional): Swap fee in basis points.
        position_intent (str, optional): Represents the desired position strategy.

    Returns:
        dict: A JSON response containing the details of the created order.
    """
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders"

    payload = {
        "symbol": symbol,
        "qty": qty,
        "notional": notional,
        "side": side,
        "type": type,
        "time_in_force": time_in_force,
        "limit_price": limit_price,
        "stop_price": stop_price,
        "trail_price": trail_price,
        "trail_percent": trail_percent,
        "extended_hours": extended_hours,
        "client_order_id": client_order_id,
        "order_class": order_class,
        "tags": tags,
        "take_profit": take_profit,
        "stop_loss": stop_loss,
        "commission": commission,
        "commission_type": commission_type,
        "source": source,
        "instructions": instructions,
        "subtag": subtag,
        "swap_fee_bps": swap_fee_bps,
        "position_intent": position_intent,
    }

    # Remove None values from payload
    payload = {k: v for k, v in payload.items() if v is not None}

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        print(f"Succesfully placed order {response.json()['id']} for {response.json()['position_intent']} of the symbol {response.json()['symbol']}. See details in response.")

    return response.json()


# order_response = create_order_for_client(
#     account_id="client_account_id",
#     symbol="AAPL",
#     qty="4.122",
#     side="buy",
#     type="market",
#     time_in_force="day",
#     commission=1.0,
#     commission_type="notional",
#     take_profit =  { "limit_price": "3.14" },
#     stop_loss =  {
#         "limit_price": "3",
#         "stop_price": "3"
#     }
# )
# print(order_response)

# Make trades

Define client:

In [36]:
client_id = clients_account_id["client_2"]

Invest 100 dolars in Microsoft stocks. We need to trade with "notional" argument of `create_order_for_client()`.

In [62]:
order_created_response = create_order_for_client(account_id=clients_account_id["client_1"], symbol="MSFT", notional="100", side="buy",type="market")
order_created_response

Succesfully placed order d38a6ec8-06a0-4a3f-af51-63c5b20ee3d2 for buy_to_open of the symbol MSFT. See details in response.


{'id': 'd38a6ec8-06a0-4a3f-af51-63c5b20ee3d2',
 'client_order_id': '40ccdf3d-ec70-4e2c-8bfb-7244c8f4ad92',
 'created_at': '2025-01-23T11:34:08.16383697Z',
 'updated_at': '2025-01-23T11:34:08.16520069Z',
 'submitted_at': '2025-01-23T11:34:08.16383697Z',
 'filled_at': None,
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
 'symbol': 'MSFT',
 'asset_class': 'us_equity',
 'notional': '100',
 'qty': None,
 'filled_qty': '0',
 'filled_avg_price': None,
 'order_class': 'simple',
 'order_type': 'market',
 'type': 'market',
 'side': 'buy',
 'position_intent': 'buy_to_open',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'accepted',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'commission': '0',
 'commission_type': 'notional',
 'subtag': None,
 'source': None,
 'expires_at': '2025

Check the open order

In [299]:
client_id

'edafa6db-b501-4d69-95c8-b92793a3680c'

In [302]:
get_order_by_id("dcfbefb1-cbaa-4e60-9cef-9db801b8eb13", "e4d31b70-6ed8-4f61-8c2c-79d7ba769d94")

{'id': 'e4d31b70-6ed8-4f61-8c2c-79d7ba769d94', 'client_order_id': '5282aab9-5bc7-4f30-b787-ea6f75f452fb', 'created_at': '2025-01-23T14:51:00.480350182Z', 'updated_at': '2025-01-23T14:51:00.57955921Z', 'submitted_at': '2025-01-23T14:51:00.480350182Z', 'filled_at': '2025-01-23T14:51:00.485818151Z', 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'asset_class': 'us_equity', 'notional': '300', 'qty': None, 'filled_qty': '1.333357037', 'filled_avg_price': '224.996', 'order_class': 'simple', 'order_type': 'market', 'type': 'market', 'side': 'buy', 'position_intent': 'buy_to_open', 'time_in_force': 'day', 'limit_price': None, 'stop_price': None, 'status': 'filled', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'commission': '0', 'commission_type': 'notional', 'subtag': None, 'source': None, 'expires_at':

{'id': 'e4d31b70-6ed8-4f61-8c2c-79d7ba769d94',
 'client_order_id': '5282aab9-5bc7-4f30-b787-ea6f75f452fb',
 'created_at': '2025-01-23T14:51:00.480350182Z',
 'updated_at': '2025-01-23T14:51:00.57955921Z',
 'submitted_at': '2025-01-23T14:51:00.480350182Z',
 'filled_at': '2025-01-23T14:51:00.485818151Z',
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
 'symbol': 'AAPL',
 'asset_class': 'us_equity',
 'notional': '300',
 'qty': None,
 'filled_qty': '1.333357037',
 'filled_avg_price': '224.996',
 'order_class': 'simple',
 'order_type': 'market',
 'type': 'market',
 'side': 'buy',
 'position_intent': 'buy_to_open',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'filled',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'commission': '0',
 'commission_type': 'notional',
 'subtag': 

Query all orders, closed or open, for client:

In [29]:
get_list_of_orders(client_id, status="all")

[{'id': 'f9cf7705-6aa0-4244-adc2-c76626330181',
  'client_order_id': 'e4dc11e6-33e9-4022-86ee-3f66e7a3e659',
  'created_at': '2025-01-23T10:08:13.581592056Z',
  'updated_at': '2025-01-23T10:08:13.583182506Z',
  'submitted_at': '2025-01-23T10:08:13.581592056Z',
  'filled_at': None,
  'expired_at': None,
  'canceled_at': None,
  'failed_at': None,
  'replaced_at': None,
  'replaced_by': None,
  'replaces': None,
  'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
  'symbol': 'MSFT',
  'asset_class': 'us_equity',
  'notional': '100',
  'qty': None,
  'filled_qty': '0',
  'filled_avg_price': None,
  'order_class': 'simple',
  'order_type': 'market',
  'type': 'market',
  'side': 'buy',
  'position_intent': 'buy_to_open',
  'time_in_force': 'day',
  'limit_price': None,
  'stop_price': None,
  'status': 'accepted',
  'extended_hours': False,
  'legs': None,
  'trail_percent': None,
  'trail_price': None,
  'hwm': None,
  'commission': '0',
  'commission_type': 'notional',
  'subtag': None

Query open orders:

In [28]:
get_list_of_orders(client_id, status="open")

[{'id': 'f9cf7705-6aa0-4244-adc2-c76626330181',
  'client_order_id': 'e4dc11e6-33e9-4022-86ee-3f66e7a3e659',
  'created_at': '2025-01-23T10:08:13.581592056Z',
  'updated_at': '2025-01-23T10:08:13.583182506Z',
  'submitted_at': '2025-01-23T10:08:13.581592056Z',
  'filled_at': None,
  'expired_at': None,
  'canceled_at': None,
  'failed_at': None,
  'replaced_at': None,
  'replaced_by': None,
  'replaces': None,
  'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
  'symbol': 'MSFT',
  'asset_class': 'us_equity',
  'notional': '100',
  'qty': None,
  'filled_qty': '0',
  'filled_avg_price': None,
  'order_class': 'simple',
  'order_type': 'market',
  'type': 'market',
  'side': 'buy',
  'position_intent': 'buy_to_open',
  'time_in_force': 'day',
  'limit_price': None,
  'stop_price': None,
  'status': 'accepted',
  'extended_hours': False,
  'legs': None,
  'trail_percent': None,
  'trail_price': None,
  'hwm': None,
  'commission': '0',
  'commission_type': 'notional',
  'subtag': None

Cancel open order by id:

In [47]:
cancel_open_order(client_id, order_created_response["id"])

Order '5bd4e635-d009-4e59-beaf-50e8e7dd4308' cancelled succesfully.


Cancel all open orders:

In [31]:
def cancel_all_open_orders(account_id):
    url = f"https://broker-api.sandbox.alpaca.markets/v1/trading/accounts/{account_id}/orders"

    response = requests.delete(url, headers=headers)

    if response.status_code == 207:
        succesful_cancellations = [order["id"] for order in response.json() if order["status"] == 200]
        unsuccesful_cancellations = [order["id"] for order in response.json() if order["status"] != 200]

        print(f"The following orders were cancelled: {', '.join(succesful_cancellations)}.")

        print(f"\nUnsuccesful cancellations: {', '.join(unsuccesful_cancellations)}.")

        return response.json()

Close an open position by id:

Open a position and close it after 7 days.
1. Open order. Register date.
2. Add to "positions_to_close" table.
3. Everyday, check "positions_to_close" table to check if there are any positions to close today. 
4. Issue position close order.
5. (Bonus) Monitor fullfilled price at market open and decide wether a limit price should be issued.

Monitor orders and positions. For the specific case of opening and closing positions everyday at closed market:
1. Have a table with order_id, order_date, close_date, status, intent_price, notional_amount, fulfilled_date, position_id, fulfilled_price, currency.
2. Place order. Register order_id, order_date, intent_price (that day's close price if market, or limit_price), notional_amount, status == pending.
3. Every day, check_status of non-definitive status (pending) orders and if open, then update to open and update fulfilled_date and fulfilled_price.
4. Every day, check planned_close_date == today() and status==open and close orders.
5. Start locally, but this table should be uploaded to MongoDB.
 

In [3]:
import sys
sys.path.append("..")

db_user = os.environ.get("MONGO_DB_USER")
db_pass = os.environ.get("MONGO_DB_PASS")
db_host = os.environ.get("MONGO_HOST")
database = os.environ.get("MONGO_DB_NAME")
mongodb_options = os.environ.get("MONGO_OPTIONS")

from src.support.data_load import MongoDBHandler

In [309]:
db_handle = MongoDBHandler(db_user=db_user, db_pass=db_pass, host=db_host, options=mongodb_options)

db_handle.connect_to_database(database)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
collection = db_handle.create_collection("orders")

### Encontrar información en MongoDB

De la colección, encontrar todo

In [16]:
collection.find()

# para convertir el cursor en información valiosa para nosotros podemos convertirlo a lista, como vemos a continuación
resultado_find_list = list(collection.find())
print(f"Hay {len(resultado_find_list)} órdenes en la colección:")
print(resultado_find_list[:2])

Hay 2 órdenes en la colección
[{'_id': ObjectId('67921fe2a36f4282e4172326'), 'mock_order_id': {'account_id': '12345', 'close_date': '2025-01-25T20:00:00tz...', 'status': 'PENDING', 'intent_price': '10', 'notional_amount': '100', 'fulfilled_date': None, 'position_id': None, 'fulfilled_price': None, 'currency': 'USD'}}, {'_id': ObjectId('67922032a36f4282e4172327'), 'order_id_mock12345': {'account_id': '12345', 'close_date': '2025-01-25T20:00:00tz...', 'status': 'PENDING', 'intent_price': '10', 'notional_amount': '100', 'fulfilled_date': None, 'position_id': None, 'fulfilled_price': None, 'currency': 'USD'}}]


Meter información de una orden

In [17]:
# order_id, order_date, planned_close_date, status, intent_price, notional_amount, fulfilled_date, position_id, fulfilled_price, currency.

mock_order = [{"content": {
    "alpaca_order_id": "order_12345",
    "order_date": "2025-01-18T20:00:00tz..."
    "alpaca_account_id": "account_12345",
    "planned_close_date": "2025-01-25T20:00:00tz...",
    "status": "PENDING",
    "intent_price": "10",
    "notional_amount": "100",
    "fulfilled_date": None,
    "position_id": None,
    "fulfilled_price": None,
    "currency": "USD"
}}]

In [18]:
db_handle.insert_documents(collection_name="orders", documents=mock_order)

## Gestionar órdenes de la estrategia

1. **Crear orden**: Crearla en alpaca y registrarla en mongoDB.
2. **Actualizar estado de órdenes**: Comprobar estado en Alpaca y si es distinto actualizar en MongoDB.
3. **Cerrar órdenes**: Comprobar en MongoDB, cerrarlas en Alpaca, comprobar que el cierre ha sido exitoso y actualizar en MongoDB. Hacer la función de comprobación y cierre flexible y atómica en cuanto a input para poder reutilizarla a modo de demo sin que sea diario por la noche. 

In [38]:
order_created_response

{'id': 'd5fab616-00c0-46aa-bb05-99aaad45f8fa',
 'client_order_id': 'bd80db97-d5e5-4244-890c-81e2ef3a0061',
 'created_at': '2025-01-23T11:08:14.266973241Z',
 'updated_at': '2025-01-23T11:08:14.268501271Z',
 'submitted_at': '2025-01-23T11:08:14.266973241Z',
 'filled_at': None,
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': 'b6d1aa75-5c9c-4353-a305-9e2caa1925ab',
 'symbol': 'MSFT',
 'asset_class': 'us_equity',
 'notional': '100',
 'qty': None,
 'filled_qty': '0',
 'filled_avg_price': None,
 'order_class': 'simple',
 'order_type': 'market',
 'type': 'market',
 'side': 'buy',
 'position_intent': 'buy_to_open',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'accepted',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'commission': '0',
 'commission_type': 'notional',
 'subtag': None,
 'source': None,
 'expires_at': '2

1 - Crear y registrar orden

In [ ]:
from dateutil import parser

# Example string
date_string = '2025-01-23T12:10:06.272064904Z'

# Parse the string
parsed_date = parser.isoparse(date_string)
print(parsed_date)

2025-01-23


In [378]:
def create_and_register_order(account_id: str,
                                symbol: str,
                                notional_amount: Union[str, float],
                                intent_open_price: float,
                                position_intent: str,
                                closure_date: datetime,
                                # **argumentos_adicionales
                                ):
    
    order_response = create_order_for_client(account_id=account_id, symbol=symbol, notional=notional_amount, position_intent=position_intent) #, **argumentos_adicionales) # Could need additional parameters

    if order_response.get("id",None) == None:
        print(f"Order could not be placed due to {order_response.get('message', '_missing message_')}:")
        print(order_response)

    order_mongodb_record = [{"content": {
        "alpaca_order_id_open": order_response["id"],
        "alpaca_order_id_close": None,
        "alpaca_account_id": account_id,
        "asset_id": order_response["asset_id"],
        "order_creation_date": parser.isoparse(order_response["created_at"]),
        "planned_close_date": closure_date,
        "symbol": symbol,
        "position_intent": position_intent,
        "status_open": "PENDING",
        "status_close": None,
        "intent_open_price": intent_open_price,
        "notional_amount": notional_amount,
        "qty": None,
        "fulfilled_open_date": None,
        "fulfilled_open_price": None,
        "fulfilled_close_date": None,
        "fulfilled_close_price": None,
        "pct_profit_trade": None,
        "currency": "USD"
    }}]

    insertion_result = db_handle.insert_documents(collection_name="orders", documents=order_mongodb_record)

    if insertion_result.inserted_ids:
        print("Document inserted successfully with _id:", insertion_result.inserted_ids)
    else:
        print("Insertion failed.")
    
    return insertion_result.inserted_ids, order_mongodb_record[0]

In [379]:
clients_account_id

{'client_1': 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13',
 'client_2': 'edafa6db-b501-4d69-95c8-b92793a3680c'}

- NOTA IMPORTANTE: habría que ajustar bien la asignación de fecha-hora de cierre. Lo ideal para este caso es cuando el precio no se mueva. Tener función de comprobación del cierre de mercado (a lo mejor es un detalle de next steps.)

In [380]:
closure_date=(datetime.today() + timedelta(days=7)).replace(tzinfo=timezone.utc)
closure_date.date()

datetime.date(2025, 1, 30)

In [381]:
inserted_id, order_mongodb_record = create_and_register_order(clients_account_id["client_1"], 
                                                              symbol="AAPL", 
                                                              notional_amount="300", 
                                                              intent_open_price="223.83",
                                                              position_intent="buy_to_open",
                                                              closure_date=closure_date)

Succesfully placed order 23e7ec13-df1a-4690-b49b-f4ad7ba2b1df for buy_to_open of the symbol AAPL. See details in response.
Document inserted successfully with _id: [ObjectId('67927386a36f4282e4172340')]


1.1 - Consultar orden por id de Alpaca

In [382]:
# These 2 are quivalent:
# first mode
collection.find_one({'content.alpaca_order_id_open': order_mongodb_record["content"]["alpaca_order_id_open"]})

# explicit mode
collection.find_one({'content.alpaca_order_id_open': {"$eq":order_mongodb_record["content"]["alpaca_order_id_open"]}})

# # By MongoDB objectID it would be
# list(collection.find({'_id':ObjectId(inserted_id)}))

{'_id': ObjectId('67927386a36f4282e4172340'),
 'content': {'alpaca_order_id_open': '23e7ec13-df1a-4690-b49b-f4ad7ba2b1df',
  'alpaca_order_id_close': None,
  'alpaca_account_id': 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13',
  'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
  'order_creation_date': datetime.datetime(2025, 1, 23, 16, 51, 17, 583000),
  'planned_close_date': datetime.datetime(2025, 1, 30, 17, 51, 17, 361000),
  'symbol': 'AAPL',
  'position_intent': 'buy_to_open',
  'status_open': 'PENDING',
  'status_close': None,
  'intent_open_price': '223.83',
  'notional_amount': '300',
  'qty': None,
  'fulfilled_open_date': None,
  'fulfilled_open_price': None,
  'fulfilled_close_date': None,
  'fulfilled_close_price': None,
  'pct_profit_trade': None,
  'currency': 'USD'}}

2 - Actualizar estado de la orden


In [383]:
def update_order_field(direction:str, # this specifices which alpaca_order_id to use for update
                       alpaca_order_id: str, field_to_update: str, new_field_value: str):
    # update status
    db_handle.db["orders"].update_one(
        {f'content.alpaca_order_id_{direction}': alpaca_order_id}, # Filter
        {"$set": {f'content.{field_to_update}': new_field_value}}  # Update
    )


2.1 - Chequear orden en mongo: Lista de órdenes en PENDING

In [384]:
pending_orders = list(collection.find({'content.status_open':'PENDING'}))
pending_orders

[{'_id': ObjectId('67927386a36f4282e4172340'),
  'content': {'alpaca_order_id_open': '23e7ec13-df1a-4690-b49b-f4ad7ba2b1df',
   'alpaca_order_id_close': None,
   'alpaca_account_id': 'dcfbefb1-cbaa-4e60-9cef-9db801b8eb13',
   'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
   'order_creation_date': datetime.datetime(2025, 1, 23, 16, 51, 17, 583000),
   'planned_close_date': datetime.datetime(2025, 1, 30, 17, 51, 17, 361000),
   'symbol': 'AAPL',
   'position_intent': 'buy_to_open',
   'status_open': 'PENDING',
   'status_close': None,
   'intent_open_price': '223.83',
   'notional_amount': '300',
   'qty': None,
   'fulfilled_open_date': None,
   'fulfilled_open_price': None,
   'fulfilled_close_date': None,
   'fulfilled_close_price': None,
   'pct_profit_trade': None,
   'currency': 'USD'}}]

2.2 - Actualizar ordenes pending por id

In [385]:
def refresh_pending_status_open(account_id, alpaca_order_id):

    # get order info from alpaca
    order_in_alpaca = get_order_by_id(account_id, alpaca_order_id)
    
    # update status field
    update_order_field(order_in_alpaca["id"], "status_open", order_in_alpaca["status"])

    # if filled, update price, date and quantity
    if order_in_alpaca["status"] == "filled":
        # update filled_price
        update_order_field(direction="open", 
                            alpaca_order_id=order_in_alpaca["id"], 
                            field_to_update="fulfilled_open_price", 
                            new_field_value=order_in_alpaca["filled_avg_price"])
        # update filled_date
        update_order_field(direction="open", 
                            alpaca_order_id=order_in_alpaca["id"], 
                            field_to_update="fulfilled_open_date", 
                            new_field_value=order_in_alpaca["filled_at"])
        # update qty
        update_order_field(direction="open", 
                            alpaca_order_id=order_in_alpaca["id"], 
                            field_to_update="qty", 
                            new_field_value=order_in_alpaca["filled_qty"])

In [386]:
def refresh_pending_status(direction:str ,account_id, alpaca_order_id):

    # get order info from alpaca
    order_in_alpaca = get_order_by_id(account_id, alpaca_order_id)
    
    # update status field
    update_order_field(direction=direction,
                       alpaca_order_id=order_in_alpaca["id"], 
                       field_to_update=f"status_{direction}", 
                       new_field_value=order_in_alpaca["status"])

    # if filled, update price, date and quantity
    if order_in_alpaca["status"] == "filled":
        # update filled_price
        update_order_field(direction = direction, 
                            alpaca_order_id = order_in_alpaca["id"], 
                            field_to_update = f"fulfilled_{direction}_price", 
                            new_field_value = order_in_alpaca["filled_avg_price"])
        # update filled_date
        update_order_field(direction = direction, 
                            alpaca_order_id = order_in_alpaca["id"], 
                            field_to_update = f"fulfilled_{direction}_date", 
                            new_field_value = order_in_alpaca["filled_at"])
        
        if direction == "open":
            # update qty
            update_order_field(direction = direction, 
                                alpaca_order_id = order_in_alpaca["id"], 
                                field_to_update ="qty", 
                                new_field_value = order_in_alpaca["filled_qty"])
        else: # close
            # update pct_profit
            filled_open_price = float(db_handle.db["orders"].find_one({f'content.alpaca_order_id_close': alpaca_order_id})["content"]["fulfilled_open_price"])
            filled_closed_price = float(order_in_alpaca["filled_avg_price"])
            pct_profit_trade = filled_closed_price / filled_open_price - 1
            
            update_order_field(direction = direction, 
                                alpaca_order_id = order_in_alpaca["id"], 
                                field_to_update = "pct_profit_trade", 
                                new_field_value = pct_profit_trade)

2.3 Pasar ids de funciones pending a la función de actualizar status pending

In [387]:
def refresh_pending_statuses_open(account_id):
    
    pending_orders_mongodb = list(collection.find({'content.status_open':'PENDING'}))

    for order in pending_orders_mongodb:
        order = order["content"]
        refresh_pending_status_open(account_id, order["alpaca_order_id_open"])

In [388]:
def refresh_pending_statuses(direction:str, account_id):
    
    pending_orders_mongodb = list(collection.find({f'content.status_{direction}':'PENDING'}))

    for order in pending_orders_mongodb:
        order = order["content"]
        refresh_pending_status(direction=direction, account_id=account_id, alpaca_order_id=order[f"alpaca_order_id_{direction}"])

Comprobando:

In [389]:
refresh_pending_statuses(direction="open", account_id=clients_account_id["client_1"])

{'id': '23e7ec13-df1a-4690-b49b-f4ad7ba2b1df', 'client_order_id': '7b6547a7-ef20-433f-918d-c7cb54a3ba21', 'created_at': '2025-01-23T16:51:17.583864329Z', 'updated_at': '2025-01-23T16:51:17.630155498Z', 'submitted_at': '2025-01-23T16:51:17.583864329Z', 'filled_at': '2025-01-23T16:51:17.611262201Z', 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'asset_class': 'us_equity', 'notional': '300', 'qty': None, 'filled_qty': '1.338473069', 'filled_avg_price': '224.136', 'order_class': 'simple', 'order_type': 'market', 'type': 'market', 'side': 'buy', 'position_intent': 'buy_to_open', 'time_in_force': 'day', 'limit_price': None, 'stop_price': None, 'status': 'filled', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'commission': '0', 'commission_type': 'notional', 'subtag': None, 'source': None, 'expires_at'

In [390]:
# refresh_pending_statuses_open(clients_account_id["client_1"])

3. **Cerrar órdenes**: Comprobar en MongoDB, cerrarlas en Alpaca, comprobar que el cierre ha sido exitoso y actualizar en MongoDB. Hacer la función de comprobación y cierre flexible y atómica en cuanto a input para poder reutilizarla a modo de demo sin que sea diario por la noche. 

1 - Comprobar en Mongo qué órdenes o posiciones se deben cerrar (habría que ajustar bien la asignación de fecha-hora de cierre)

In [371]:
closing_datetime = datetime(2025, 1, 30)
# Comprobar qué órdenes coinciden ya han pasado esa fecha y siguen abiertas
def check_open_due_orders(closing_datetime: datetime):
    due_order_list_mongodb = list(db_handle.db["orders"].find(
                {"$and":
                [{'content.planned_close_date': {"$gte": closing_datetime}}, # After closing time filter
                {'content.status_open': 'filled'}
                ]}))
    
    due_order_list_mongodb = [due_order["content"] for due_order in due_order_list_mongodb]
    
    return due_order_list_mongodb

2.1 - Update status closed in database and close qty of position equal to quantity of closing order

In [391]:
mongodb_order_dict = db_handle.db["orders"].find_one({"content.alpaca_order_id_open": "23e7ec13-df1a-4690-b49b-f4ad7ba2b1df"})
mongodb_order_dict = mongodb_order_dict["content"]

In [392]:
# response_close = close_symbol_position_client(account_id=mongodb_order_dict["alpaca_account_id"],
#                                  symbol=mongodb_order_dict["symbol"], 
#                                  magnitude=mongodb_order_dict["qty"], 
#                                  mode = "qty")

In [393]:
get_order_by_id(mongodb_order_dict["alpaca_account_id"],"d1cdb6b1-4b35-4c17-abf0-094705853ceb")

{'id': 'd1cdb6b1-4b35-4c17-abf0-094705853ceb', 'client_order_id': 'd5485de7-6796-47ac-a085-5a596e614b6e', 'created_at': '2025-01-23T15:43:34.596902789Z', 'updated_at': '2025-01-23T15:43:34.686180197Z', 'submitted_at': '2025-01-23T15:43:34.605749963Z', 'filled_at': '2025-01-23T15:43:34.636741242Z', 'expired_at': None, 'canceled_at': None, 'failed_at': None, 'replaced_at': None, 'replaced_by': None, 'replaces': None, 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'asset_class': 'us_equity', 'notional': None, 'qty': '1.333357037', 'filled_qty': '1.333357037', 'filled_avg_price': '226.17', 'order_class': '', 'order_type': 'market', 'type': 'market', 'side': 'sell', 'position_intent': 'sell_to_close', 'time_in_force': 'day', 'limit_price': None, 'stop_price': None, 'status': 'filled', 'extended_hours': False, 'legs': None, 'trail_percent': None, 'trail_price': None, 'hwm': None, 'subtag': None, 'source': None, 'expires_at': '2025-01-23T21:00:00Z'}


{'id': 'd1cdb6b1-4b35-4c17-abf0-094705853ceb',
 'client_order_id': 'd5485de7-6796-47ac-a085-5a596e614b6e',
 'created_at': '2025-01-23T15:43:34.596902789Z',
 'updated_at': '2025-01-23T15:43:34.686180197Z',
 'submitted_at': '2025-01-23T15:43:34.605749963Z',
 'filled_at': '2025-01-23T15:43:34.636741242Z',
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
 'symbol': 'AAPL',
 'asset_class': 'us_equity',
 'notional': None,
 'qty': '1.333357037',
 'filled_qty': '1.333357037',
 'filled_avg_price': '226.17',
 'order_class': '',
 'order_type': 'market',
 'type': 'market',
 'side': 'sell',
 'position_intent': 'sell_to_close',
 'time_in_force': 'day',
 'limit_price': None,
 'stop_price': None,
 'status': 'filled',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'subtag': None,
 'source': None,
 'expires_at': '2025-01-

In [394]:
def close_order_position(mongodb_order_dict: dict):
    # close qty amount in corresponding asset position
    response_close = close_symbol_position_client(account_id=mongodb_order_dict["alpaca_account_id"],
                                 symbol=mongodb_order_dict["symbol"], 
                                 magnitude=mongodb_order_dict["qty"], 
                                 mode = "qty")
    
    # initialize close order status
    update_order_field(direction="open", # IMPORTANT: direction="open" to use alpaca_order_id_open
                       alpaca_order_id = mongodb_order_dict["alpaca_order_id_open"],
                        field_to_update = "status_close", 
                        new_field_value = "PENDING")
    
    # update alpaca_order_id_close
    update_order_field(direction="open", # IMPORTANT: direction="open" to use alpaca_order_id_open
                       alpaca_order_id = mongodb_order_dict["alpaca_order_id_open"],
                        field_to_update = "alpaca_order_id_close", 
                        new_field_value = response_close["id"])
    
    return response_close
    

In [395]:
response_close = close_order_position(mongodb_order_dict)

{'code': 40310100, 'message': 'trade denied due to pattern day trading protection'}


KeyError: 'id'

In [396]:
refresh_pending_statuses(direction="close", account_id=clients_account_id["client_1"])

{'code': 40010001, 'message': 'order_id is missing'}


KeyError: 'id'

2.2 - Close order

In [245]:
get_open_positions_client("edafa6db-b501-4d69-95c8-b92793a3680c")

[{'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415', 'symbol': 'AAPL', 'exchange': 'NASDAQ', 'asset_class': 'us_equity', 'asset_marginable': True, 'qty': '1', 'avg_entry_price': '234.59', 'side': 'long', 'market_value': '224.4', 'cost_basis': '234.59', 'unrealized_pl': '-10.19', 'unrealized_plpc': '-0.0434374866788866', 'unrealized_intraday_pl': '0.57', 'unrealized_intraday_plpc': '0.0025465755260689', 'current_price': '224.4', 'lastday_price': '223.83', 'change_today': '0.0025465755260689', 'qty_available': '1'}]


[{'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
  'symbol': 'AAPL',
  'exchange': 'NASDAQ',
  'asset_class': 'us_equity',
  'asset_marginable': True,
  'qty': '1',
  'avg_entry_price': '234.59',
  'side': 'long',
  'market_value': '224.4',
  'cost_basis': '234.59',
  'unrealized_pl': '-10.19',
  'unrealized_plpc': '-0.0434374866788866',
  'unrealized_intraday_pl': '0.57',
  'unrealized_intraday_plpc': '0.0025465755260689',
  'current_price': '224.4',
  'lastday_price': '223.83',
  'change_today': '0.0025465755260689',
  'qty_available': '1'}]